Финальный проект:
Проект осуществляется на данных из вебинара 

Целевая метрика - money precision @ 5. Порог для уcпешной сдачи проекта money precision @ 5 > 20%

Бизнес ограничения в топ-5 товарах:
- Для каждого юзера 5 рекомендаций (иногда модели могут возвращать < 5)
- 2 новых товара (юзер никогда не покупал)
- 1 дорогой товар, > 7 долларов
- Все товары из разных категорий (категория - sub_commodity_desc)

Стоимость каждого рекомендованного товара > 1 доллара. Будет  будет private тестовый датасет для измерения финального качества Не обязательно использовать 2-ух уровневые рекоммендательные системы в проекте. Сдается  код проекта в виде github репозитория и .csv файл с рекомендациями. В .csv файле 2 столбца: user_id - (item_id1, item_id2, ..., item_id5)


In [1]:
import lightgbm
lightgbm.__version__

'3.0.0'

In [2]:
import implicit
implicit.__version__

'0.4.2'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
import re
import time
from lightgbm import LGBMClassifier
import os, sys
#import src.utils
#from src.metrics import precision_at_k
from implicit.nearest_neighbours import ItemItemRecommender, bm25_weight 
# from src.utils import prefilter_items
# from src.recommenders import MainRecommender

**Для удобства положил класс MainRecommenderздесь

In [4]:
#comment
class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, prices, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['sales_value'].count().reset_index()
        self.top_purchases.sort_values('sales_value', ascending=False, inplace=True)

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['sales_value'].count().reset_index()
        self.overall_top_purchases.sort_values('sales_value', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()
        
        # Частотность  покупок товара 
        self.n_users = data['user_id'].nunique()
        self.purchases_frequency = data.groupby('item_id')['user_id'].nunique().reset_index() 
        self.purchases_frequency.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
        self.purchases_frequency['share_unique_users']=self.purchases_frequency['share_unique_users']/self.n_users
        self.top_frequency  = self.purchases_frequency[self.purchases_frequency['share_unique_users'] >= 
                                         np.quantile(self.purchases_frequency['share_unique_users'], QUANT_FREQ)]
        self.top_frequency.sort_values('share_unique_users', ascending=False, inplace=True)
        self.top_frequency= self.top_frequency.item_id.tolist()
        #self.data = data[data['item_id'].isin(top_frequency)]
                
        self.user_item_matrix, self.matrix_index, self.matrix_columns = self._prepare_matrix(data)  # pd.DataFrame
        
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T, K1=4,B=0.165).T

        self.model = self.fit(self.user_item_matrix)
#         self.bpr = self.fit_bpr(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
        self.item_factors = self.model.item_factors
        self.user_factors = self.model.user_factors
#         self.price_list = prices
        
        self.items_emb_df, self.users_emb_df = self.get_embeddings(self)
        
        
    @staticmethod
    def get_embeddings(self):
        items_emb = self.item_factors
        items_emb_df = pd.DataFrame(items_emb)
        items_emb_df.reset_index(inplace=True)
        items_emb_df['item_id'] = items_emb_df['index'].apply(lambda x: self.id_to_itemid[x])
        items_emb_df = items_emb_df.drop('index', axis=1)

        users_emb = self.user_factors
        users_emb_df = pd.DataFrame(users_emb)
        users_emb_df.reset_index(inplace=True)
        users_emb_df['user_id'] = users_emb_df['index'].apply(lambda x: self.id_to_userid[x])
        users_emb_df = users_emb_df.drop('index', axis=1)

        return items_emb_df, users_emb_df
        

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', 
                                          columns='item_id',
                                          values='sales_value',
                                          aggfunc='count',
                                          fill_value=0
                                          )
        matrix_index = user_item_matrix.index
        matrix_columns = user_item_matrix.columns

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit
        return user_item_matrix, matrix_index, matrix_columns
    

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
    

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
        
        own_recommender = ItemItemRecommender(K=1)  # Кол-во ближайших соседей
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        return own_recommender
    

    @staticmethod
    def fit(user_item_matrix, n_factors=32, regularization=0.001, iterations=20, num_threads=8):
        
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        return model
    
    @staticmethod
    def fit_bpr(user_item_matrix, n_factors=31, regularization=0.001, iterations=20, num_threads=8):
        
        bpr = BayesianPersonalizedRanking(factors=n_factors,
                                             regularization=regularization,
                                             iterations=iterations,
                                             num_threads=num_threads)
        
        bpr.fit(csr_matrix(user_item_matrix).T.tocsr())
        return bpr

    def _update_dict(self, user_id):
        
        """Если появился новый user / item, то нужно обновить словари"""
        
        if user_id not in self.userid_to_id.keys():
            print(f"user_id: '\033[94m{user_id}\033[0m' not in dict, add...")
            max_id = max(list(self.userid_to_id.values()))
            max_id += 1
            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})
            

    def _get_similar_item(self, item_id):
        
        """Находит товар, похожий на item_id"""
        
        # Товар похож на себя -> рекомендуем 2 товара
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]
    

    def _extend_with_top_popular(self, recommendations, N=5):
        
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""
        
        if len(recommendations) < N:
            recommendations.extend(self.top_frequency[:N]) #overall_top_purchases[:N])
            recommendations = recommendations[:N]
        return recommendations
    

    def _get_recommendations(self, user, model, N=5):
        
        """Рекомендации через стардартные библиотеки implicit"""
        
        
        if user not in self.userid_to_id.keys():
            res = self._extend_with_top_popular([], N=N)
        else:         
            res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                            user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                            N=N,
                                            filter_already_liked_items=False,
                                            filter_items=None,
                                            recalculate_user=True)]
            res = self._extend_with_top_popular(res, N=N)
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    

    def get_als_recommendations(self, user, N=5):
        
        """Рекомендации через стардартные библиотеки implicit"""
        
        return self._get_recommendations(user, model=self.model, N=N)
    
    
    def get_bayesian_recommendations(self, user, N=5):
        return self._get_recommendations(user, model=self.bpr, N=N)

    
    def get_own_recommendations(self, user, N=5):
        
        """Рекомендуем товары среди тех, которые юзер уже купил"""
     
        return self._get_recommendations(user, model=self.own_recommender, N=N)
    

    def get_similar_items_recommendation(self, user, N=5):
        
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        
        if user not in self.userid_to_id.keys():
            res = self._extend_with_top_popular([], N=N)
        else:
        
            top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

            res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
            res = self._extend_with_top_popular(res, N=N)
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    

    def get_similar_users_recommendation(self, user, N=5):
        
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        
        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))


        res = self._extend_with_top_popular(res, N=N)
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [5]:
DATA_PATH = './data/retail_train.csv'
TEST_PATH = './data/retail_test1.csv'
ITEM_FEATURES_PATH = './data/product.csv'
USER_FEATURES_PATH = './data/hh_demographic.csv'
SPLIT_WEEK = 3
N_POPULAR_ITEMS =20000
DEL_LAST_WEEK = 90
INIT_NUM_RECS = 500
N_FIN_RECS = 5
QUANT_FREQ = 0.55
# NUM_THREADS = 8

In [6]:
def get_raw_data(data_path):
    
    """Загружаем данные"""
    
    print(f'Preparing raw data...{data_path} ', end='')
    data = pd.read_csv(data_path)
    print(f'Number of items \033[32m{len(data)}\033[0m')
    print(f'Уникальных товаров {len(data.item_id.unique())}')
    print(f'Уникальных user-ов {len(data.user_id.unique())}')
    return data

def get_split_data(data, week=SPLIT_WEEK):
    
    """Делим данные на train и val """
    
    print(f"Делим данные с {data['week_no'].min()} по {data['week_no'].max() - week} неделю ->train")
    train = data[data['week_no'] <= data['week_no'].max() - week]
    val = data[data['week_no'] > data['week_no'].max() - week]
    print(f'Number of items in  train \033[32m{len(train)}\033[0m,  val  \033[32m{len(val)}\033[0m')
    print(f'Уникальных товаров в train {len(train.item_id.unique())}')
    print(f'Уникальных user-ов в train {len(train.user_id.unique())}')
    if week!=0 : 
        print(f"и с  {val['week_no'].min()}  по {data['week_no'].max()}->val ")
        print(f'Уникальных товаров  в  val {len(val.item_id.unique())}')
        print(f'Уникальных user-ов в val {len(val.user_id.unique())}')
    return train, val
    
def get_preprared_features(item_features_path, user_features_path):
    
    """Загружаем данные product.csv и hh_demographic.csv""" 
    
    print(f"Preparing item's features...{item_features_path} ", end='')
    item_features = pd.read_csv(item_features_path)
    item_features.columns = [col.lower() for col in item_features.columns]
    item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
     # encode commodities in item_features
    item_features['sub_commodity_desc'] = pd.Categorical(item_features['sub_commodity_desc'])
    item_features['sub_commodity_desc_code'] = item_features['sub_commodity_desc'].cat.codes
    print(f'Number of items \033[32m{len(item_features)}\033[0m')

    print(f"Preparing user's features...{user_features_path} ", end='')
    user_features = pd.read_csv(user_features_path)
    user_features.columns = [col.lower() for col in user_features.columns]
    user_features.rename(columns={'household_key': 'user_id'}, inplace=True)
    print(f'Number of items \033[32m{len(user_features)}\033[0m')
    return item_features, user_features

def prefilter_items(data, take_n_popular=N_POPULAR_ITEMS, del_last_week=DEL_LAST_WEEK, margin_slice_rate=0.8):
    
    """Предфильтрация товаров"""
    
    n_before = data['item_id'].nunique()
    
    print(' Сейчас удалим записи quantity=0, было записей: ', len(data), end = '')
    data = data.drop(data[data['quantity']==0].index)
    print(f' Осталось \033[32m{len(data)}\033[0m')
    
    print(' Сейчас удалим записи sales_value=0, было записей: ', len(data), end = '')
    data = data.drop(data[data['sales_value']==0].index)
    print(f' Осталось \033[32m{len(data)}\033[0m')

    print(' Создаем price = sales_value / quantity')
    data['price'] = data['sales_value'] / data['quantity']
    
    print(' Сейчас удалим записи price<1, было записей: ', len(data), end = '')
    data = data[data['price'] > 1]
    print(f' Осталось \033[32m{len(data)}\033[0m')

    print(' Сейчас удалим записи price>25, было записей: ', len(data), end = '')
    data = data[data['price'] < 25]
    print(f' Осталось \033[32m{len(data)}\033[0m')
    
    
    
    
    # !!!! 
    n_users = data['user_id'].nunique()
    purchases_frequency = data.groupby('item_id')['user_id'].nunique().reset_index() 
    purchases_frequency.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    purchases_frequency['share_unique_users']=purchases_frequency['share_unique_users']/n_users
    qv = np.quantile(purchases_frequency["share_unique_users"], QUANT_FREQ)
    print(f' Сейчас удалим записи о товарах с  purchases_frequency < {qv}, было записей:  {len(data)}', end = '')
    top_frequency  = purchases_frequency[purchases_frequency['share_unique_users'] >= 
                                         np.quantile(purchases_frequency['share_unique_users'], QUANT_FREQ)].item_id.tolist()
    data = data[data['item_id'].isin(top_frequency)]
    print(f' Осталось \033[32m{len(data)}\033[0m')
          
          
    
    print(f' Сейчас удалим записи о товарах которые не продавались послeдние {DEL_LAST_WEEK} недели: ', len(data), end = '')
    purchases_last_week = data.groupby('item_id')['week_no'].max().reset_index()
    weeks = purchases_last_week[
        purchases_last_week['week_no'] > data['week_no'].max() - DEL_LAST_WEEK].item_id.tolist()
    data = data[data['item_id'].isin(weeks)]
    print(f' Осталось \033[32m{len(data)}\033[0m')
    
    # Удаление 10% товаров c наименьшей выручкой (сдвигает минимум выручки с 1.1$ до 94.8$ для unsplitted data)
#     marginality = data.groupby('item_id')['sales_value'].sum().reset_index()
#     ten_percent_slice_idx = int(marginality.shape[0] * margin_slice_rate)

#     top_margin = marginality.sort_values('sales_value', ascending=False)[:ten_percent_slice_idx].item_id.tolist()
#     data = data[data['item_id'].isin(top_margin)]
    
    # Выбор топ-N самых популярных товаров (N = take_n_popular)
#     print(f' Сейчас удалим записи о товарах не попавших в список популярных top{take_n_popular}: ', len(data), end = '')
#     popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
#     top_popular = popularity.sort_values('sales_value', ascending=False)[:take_n_popular].item_id.tolist()
#     data = data[data['item_id'].isin(top_popular)]
#     print(f' Осталось \033[32m{len(data)}\033[0m')
    
    print(f"Items variety reduced from: {n_before} to: {data['item_id'].nunique()} samples...") 
    return data

def get_top_more_4(df):
    ## Почитаем частоту покупок товаров  >4  
    df = df.copy()
    df = df[df['price_avg']>4]
    df = df.groupby(['item_id','price_avg'])['user_id'].nunique().reset_index() 
    df.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    df['share_unique_users']=df['share_unique_users']/n_users
    top_more_4  = df[df['share_unique_users'] >= np.quantile(df['share_unique_users'], 0.75)]  
    top_more_4.sort_values('share_unique_users', ascending=False, inplace=True)
    print(top_more_4.head(10))
    top_more_4 = top_more_4.item_id.tolist()
    return top_more_4

def get_top_more_4_for_user_dict(df):

    res = {}

    df = data_train.copy()
    df = df[df['price']>4]

    uisq4 = df.groupby(['user_id'])['quantity'].sum().reset_index()  # всего товаров  price>4 куплено user-ом 
    uisq4.rename(columns={'quantity': 'all_qnt4'}, inplace=True)
                                                                
    # добавляем эти данные в датафрейм для расчета:

    uifr4 = df.groupby(['user_id', 'item_id'])['quantity'].sum().reset_index()
    uifr4 =  uifr4.merge(uisq4, on=['user_id'], how='left')
    uifr4['freq4'] = uifr4['quantity']/uifr4['all_qnt4']
    uifr4
    
    for user_id in df['user_id'].unique():
        top4 = uifr4[uifr4['user_id']==user_id]
        top4.sort_values('freq4', ascending=False, inplace=True)
        res.update({user_id:top4['item_id'].tolist()})
    res
        
        
    return res

def get_price_list(data_1, _id='item_id', target='price'):
    
    """
    Used 4 creating check dict (price list)
    From all historical data
    Can be used once and stored then updated if needs
    """
    
    pl_1 = data_1.groupby(_id)[target].mean().reset_index()
    #pl_2 = data_2.groupby(_id)[target].mean().reset_index()
    d1 = dict(zip(pl_1[_id], pl_1[target]))
    #d2 = dict(zip(pl_2[_id], pl_2[target]))
   # pl_emb = {**d1, **d2}
    
    return d1 #pl_emb   # Возвращаем словарь id_items : avg(price), где все id_items из train и val, 
                    # но цена из val (если таковая там блыа)

def get_bought_ever_list(data_1, _id='user_id', target='item_id'):
    
    """
    Used 4 creating check dict (user's ever bought list)
    From all historical data
    Can be used once and stored then updated if needs
    """
    
    pl_1 = data_1.groupby(_id)[target].unique().reset_index()
    d1 = dict(zip(pl_1[_id], pl_1[target]))
    
    return d1

def get_item_commodities_list(feats, _id='item_id', target='sub_commodity_desc_code'):
    
    """
    Used 4 creating check dict (item_id - commodity_type)
    From all historical data
    Can be used once and stored then updated if needs
    """
    
    res = dict(zip(feats[_id], feats[target]))
    return res

def get_users_purchases_frequency(data, _id='item_id', target='basket_id' ):
    d1={}
    for id_user in data['user_id'].unique():
        n_basket = data[data['user_id']== id_user]['basket_id'].nunique()
        df = data[data['user_id']== id_user]
        purchases_frequency = df.groupby('item_id')['basket_id'].nunique().reset_index() 
        purchases_frequency['basket_id']= purchases_frequency['basket_id']/n_basket
        purchases_frequency.sort_values('basket_id', ascending=False, inplace=True)
        purchases_frequency=purchases_frequency.item_id.tolist()
        d1.update({id_user:purchases_frequency})
    return d1    
    
def m1(recommended_list, bought_list, prices):
    precision = np.isin(bought_list, recommended_list)
    return precision.sum()

def money_precision_at_k(recommended_list, bought_list, prices, k=5):
    bought_list = np.array(bought_list)
    if recommended_list == 0:
        recommended_list = np.array(top_more_4[:5])
    else:
        recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array([prices[i] for i in recommended_list])
   # print('prices_recommended', prices_recommended)
    prices_bought=np.array([itemid_to_price[i] if i in itemid_to_price.keys() else 0   for i in bought_list])
    #prices_bought = np.array([prices[i] for i in bought_list])
  #  print('prices_bought',prices_bought.shape)
    flags = np.isin(bought_list, recommended_list)
    #print('flags',flags.shape)
    precision = np.dot(flags, prices_bought) / np.dot(np.ones(k), prices_recommended)
    return precision


## Функции для проверки бизнес правил

def res_list(recommended_list, bought_list):
    l = []
    for i, rec in enumerate(recommended_list,0):
        #print(rec)
        if rec in bought_list: l.append(1)
        else:l.append(0) 
    return np.array(l)

def price_more_7(recommended_list, prices ):     
    prices_recommended = [prices[i] for i in recommended_list]
    res = [1 if p>7  else 0 for p in prices_recommended]
    return sum(res)

def more_2(recommended_list, users_bought_list, user_id):  
    if user_id in users_bought_list.keys():
        user_bought_list=users_bought_list[user_id]
    else: user_bought_list=[]
    #print(len(user_bought_list))
    flags = np.isin(recommended_list, user_bought_list)
    return sum(flags)

def dep_5(recommended_list, item_to_commodity):    
    d=np.array([item_to_commodity[item] if item in item_to_commodity.keys() else 0   for item in recommended_list])
    return len(np.unique(d))


In [7]:
%%time
raw_data = get_raw_data(DATA_PATH)
test_data = get_raw_data(TEST_PATH)
item_features, user_features = get_preprared_features(ITEM_FEATURES_PATH, USER_FEATURES_PATH)
raw_data

Preparing raw data..../data/retail_train.csv Number of items 2396804
Уникальных товаров 89051
Уникальных user-ов 2499
Preparing raw data..../data/retail_test1.csv Number of items 88734
Уникальных товаров 20497
Уникальных user-ов 1885
Preparing item's features..../data/product.csv Number of items 92353
Preparing user's features..../data/hh_demographic.csv Number of items 801
Wall time: 2.14 s


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [8]:
test_data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
88729,98,41944918665,684,945779,2,2.00,421,0.0,1705,98,0.0,0.0
88730,98,41944918665,684,993617,2,2.00,421,0.0,1705,98,0.0,0.0
88731,98,41944918665,684,1128647,2,2.00,421,0.0,1705,98,0.0,0.0
88732,98,41944918665,684,9526886,2,0.60,421,0.0,1705,98,0.0,0.0


In [9]:
data_train, data_val = get_split_data(raw_data, SPLIT_WEEK)

Делим данные с 1 по 92 неделю ->train
Number of items in  train 2311616,  val  85188
Уникальных товаров в train 87641
Уникальных user-ов в train 2499
и с  93  по 95->val 
Уникальных товаров  в  val 20621
Уникальных user-ов в val 1899


In [10]:
data_test, _ = get_split_data(test_data, 0)

Делим данные с 96 по 98 неделю ->train
Number of items in  train 88734,  val  0
Уникальных товаров в train 20497
Уникальных user-ов в train 1885


In [11]:
# Prefilter routine
data_train = prefilter_items(data_train, N_POPULAR_ITEMS) # Prefilter routine
#data_val_lvl_1 = prefilter_items(data_val_lvl_1, N_POPULAR_ITEMS, DEL_LAST_WEEK) # Prefilter routine

 Сейчас удалим записи quantity=0, было записей:  2311616 Осталось 2298730
 Сейчас удалим записи sales_value=0, было записей:  2298730 Осталось 2294723
 Создаем price = sales_value / quantity
 Сейчас удалим записи price<1, было записей:  2294723 Осталось 1650407
 Сейчас удалим записи price>25, было записей:  1650407 Осталось 1647190
 Сейчас удалим записи о товарах с  purchases_frequency < 0.0012009607686148918, было записей:  1647190 Осталось 1588402
 Сейчас удалим записи о товарах которые не продавались послeдние 90 недели:  1588402 Осталось 1588402
Items variety reduced from: 87641 to: 36605 samples...


In [12]:
#  _top_purchases = raw_data.groupby(['user_id', 'item_id'])['sales_value'].count().reset_index()
# _top_purchases.sort_values('sales_value', ascending=False, inplace=True)
# _top_purchases

In [13]:
n_users = data_train['user_id'].nunique()
purchases_frequency = data_train.groupby('item_id')['user_id'].nunique().reset_index() 
purchases_frequency.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
purchases_frequency['share_unique_users']=purchases_frequency['share_unique_users']/n_users
top_frequency  = purchases_frequency[purchases_frequency['share_unique_users'] >= 
                                         np.quantile(purchases_frequency['share_unique_users'], 0.75)]
# top= purchases_frequency[purchases_frequency['item_id'].isin(top_frequency)]    
top_frequency.sort_values('share_unique_users', ascending=False, inplace=True)
top_frequency= top_frequency.item_id.tolist()
top_frequency

<ipython-input-13-9e5351faa952>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_frequency.sort_values('share_unique_users', ascending=False, inplace=True)


[1082185,
 1029743,
 1127831,
 981760,
 916122,
 1004906,
 866211,
 844179,
 1044078,
 1068719,
 995242,
 961554,
 1106523,
 878996,
 1081177,
 859075,
 854852,
 923746,
 5569230,
 840361,
 1096036,
 951590,
 899624,
 845208,
 5568378,
 962568,
 965766,
 1024306,
 1005186,
 893018,
 1133018,
 903325,
 9526410,
 1006184,
 987562,
 6034857,
 1136257,
 986912,
 833025,
 1105488,
 5592931,
 1053690,
 1049788,
 871756,
 12810393,
 910032,
 1043590,
 9527290,
 1023720,
 965267,
 8090537,
 1027569,
 999858,
 8090521,
 995965,
 901062,
 847982,
 1037863,
 908531,
 1042438,
 865178,
 933835,
 5569845,
 857503,
 913785,
 985999,
 979707,
 5569471,
 962229,
 874972,
 5569327,
 5585510,
 1022254,
 832678,
 897125,
 870547,
 897954,
 834117,
 1041796,
 898068,
 1070820,
 1101173,
 1102959,
 1126899,
 909714,
 1076056,
 990797,
 1026118,
 883932,
 1056509,
 854405,
 930118,
 835098,
 1110843,
 909894,
 12301109,
 880150,
 863447,
 952163,
 1051211,
 918335,
 913210,
 940766,
 983584,
 844165,
 97619

**Генерим фичи для весов UI матрицы, все пробовал, НИЧЕГО НЕ ДАЛО ЗНАЧИМОГО ПРИРОСТА, оставил sales_value и count

In [14]:
# общее кол-во купленных пользователем товаров
tiq = data_train.groupby(['user_id'])['quantity'].sum().reset_index()
tiq.rename(columns={'quantity': 'all_qnt'}, inplace=True)
#  кол-во купленных пользователем товаров price >4
tiq4 = data_train[data_train['price']>4].groupby(['user_id'])['quantity'].sum().reset_index()
tiq4.rename(columns={'quantity': 'all_qnt4'}, inplace=True)
# колво данного товара, купленного пользователем
ui_qty = data_train.groupby(['user_id', 'item_id'])['quantity'].sum().reset_index()
ui_qty.rename(columns={'quantity': 'item_user_qnt'}, inplace=True)
# добавляем эти данные в датафрейм для расчета:
df = data_train.copy()
df = df.merge(tiq, on='user_id', how='left')
df = df.merge(tiq4, on='user_id', how='left')
df =  df.merge(ui_qty, on=['user_id', 'item_id'], how='left')
#df['freq'] = df['quantity']/df['item_user_qnt']
#df['freq4'] = df['quantity']/df['all_qnt4']
df['log_sv'] = np.log(df['sales_value'])

price_avg =df.groupby(['item_id'])['price'].mean().reset_index() 
price_avg.rename(columns={'price': 'price_avg'}, inplace=True)
price_avg
df = df.merge(price_avg, on='item_id', how='left')
#df['qnt_pr_all_qnt'] = df['quantity']*df['price_avg']/df['all_qnt']

data_train = df.copy()
#df[(df['user_id']==2370) & (df['item_id']==1004906)]
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,all_qnt,all_qnt4,item_user_qnt,log_sv,price_avg
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39,652,112.0,2,0.329304,2.516925
1,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,1.21,652,112.0,3,0.190620,1.344498
2,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50,652,112.0,1,0.405465,1.656176
3,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0,1.57,652,112.0,1,0.451076,2.388333
4,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99,652,112.0,10,1.095273,2.945308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588397,1707,41400727500,642,1139714,1,1.29,320,0.00,1524,92,0.0,0.0,1.29,1855,180.0,4,0.254642,1.242381
1588398,1707,41400727500,642,15926885,1,1.50,320,-0.99,1524,92,0.0,0.0,1.50,1855,180.0,1,0.405465,1.867129
1588399,77,41400727886,642,908064,1,2.00,320,-1.29,1625,92,0.0,0.0,2.00,1712,194.0,1,0.693147,2.333773
1588400,77,41400727886,642,10457110,1,2.00,320,-1.29,1625,92,0.0,0.0,2.00,1712,194.0,1,0.693147,2.114085


In [15]:
%%time
# подсчитаем частоту покупок товаров дороге 4$ для каждого user_id 
top_more_4_for_user_dict = get_top_more_4_for_user_dict(data_train)
print(top_more_4_for_user_dict[1704])

<ipython-input-6-dbbd52da4bbe>:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top4.sort_values('freq4', ascending=False, inplace=True)


[873324, 1211227, 1007929]
Wall time: 2.78 s


In [16]:
%%time
# Подсчитаем частоту покупок товаров  ценой >4$ 
top_more_4 = get_top_more_4(data_train)  # 

      item_id  price_avg  share_unique_users
1729   916122   4.298596            0.439151
4053  1049788   4.713854            0.233387
2575   965267   4.033313            0.224980
2906   985999   4.162482            0.200961
1041   874972   8.486103            0.198559
654    854405   6.611692            0.180945
1991   930118   4.672430            0.180144
5169  1110843   4.209955            0.178143
804    863447   6.128294            0.175741
403    839419   6.401848            0.166133
Wall time: 235 ms


<ipython-input-6-dbbd52da4bbe>:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_more_4.sort_values('share_unique_users', ascending=False, inplace=True)


In [17]:
%%time
# Подсчитаем частоту покупок товаров по каждым user_id и получим отсортированный словарь по убыванию частоты 
users_purchases_frequency = get_users_purchases_frequency(data_train, _id='item_id', target='basket_id' ) # словарь 
print(users_purchases_frequency[1704])

[1407216, 12384431, 873324, 1211227, 1276485, 1051540, 1063594, 1082310, 1092040, 1095100, 1129982, 1140177, 1155121, 1252245, 830887, 1311202, 1019479, 5592931, 6424471, 6424472, 6464126, 8119157, 9297055, 9396770, 10182803, 10182856, 1028715, 1016210, 833754, 1007929, 856166, 860975, 870720, 870826, 878290, 878445, 890610, 895743, 897306, 900418, 904359, 912860, 917174, 923746, 937276, 945914, 976835, 981760, 987562, 993592, 1004596, 1005186, 1007082, 12428436]
Wall time: 19.9 s


In [18]:
%%time
itemid_to_price = get_price_list(data_train)  # словарь прайс лист item_id  - цена
users_bought_history = get_bought_ever_list(data_train)  # словарь с  историей покупок  user_id - item's
item_to_commodity =  get_item_commodities_list(item_features)  # словарь com_id  - item's
users_bought = get_bought_ever_list(data_val)
print(len(itemid_to_price), len(users_bought_history), len(item_to_commodity), len(users_bought))

36605 2498 92353 1899
Wall time: 694 ms


In [19]:
%%time
recommender = MainRecommender(data_train, itemid_to_price)

<ipython-input-4-f0d50b57f245>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.top_frequency.sort_values('share_unique_users', ascending=False, inplace=True)



Wall time: 43.2 s


In [20]:
%%time

result_lvl_1 = data_val.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1['base_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=INIT_NUM_RECS))
result_lvl_1['als_rec'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=INIT_NUM_RECS))

Wall time: 1min 22s


In [21]:
result_lvl_1

,user_id,actual,base_rec,als_rec
0,1,"[883616, 917704, 931860, 961554, 995242, 10020...","[856942, 940947, 5577022, 934369, 9527290, 809...","[1047619, 896085, 1051211, 1075074, 948670, 86..."
1,6,"[909479, 7431990, 6553035, 12263667, 13382461,...","[8203834, 13003092, 863447, 1098844, 9911484, ...","[878996, 1082185, 1024306, 930118, 871611, 857..."
2,7,"[850841, 1085476, 12696020, 822346, 839747, 84...","[1122358, 828867, 1079023, 9338009, 6602729, 9...","[1127232, 839419, 1003188, 856335, 1127831, 92..."
3,8,"[884897, 893501, 1008074, 1051211, 6772795, 93...","[969932, 981660, 12302069, 972931, 1105433, 98...","[916122, 965766, 840361, 1068719, 1029743, 981..."
4,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[862799, 1135468, 1056005, 1040346, 1039126, 1...","[923746, 1082185, 893018, 1029743, 1096036, 11..."
...,...,...,...,...
1894,2495,[1096603],"[867931, 906838, 1067419, 5568445, 839656, 981...","[839656, 916122, 876695, 917654, 867931, 11268..."
1895,2496,[6534178],"[7441210, 995876, 820122, 1056509, 1000753, 12...","[844179, 1004906, 12810393, 916122, 1043751, 1..."
1896,2497,"[820361, 884896, 896938, 938139, 965719, 99513...","[1135834, 1031864, 870515, 8090532, 849202, 94...","[5569230, 957951, 963971, 899624, 5569471, 844..."
1897,2498,"[32439, 73048, 386608, 404300, 15722019, 15722...","[1100379, 1076580, 1110031, 10456152, 5565356,...","[995242, 1053690, 1136257, 859075, 916122, 556..."


In [22]:
result_lvl_1['MP5'] = result_lvl_1.apply(lambda row: money_precision_at_k(row['als_rec'], row['actual'], itemid_to_price, k=5), axis=1)
als =result_lvl_1['MP5'].mean()
result_lvl_1['MP5'] = result_lvl_1.apply(lambda row: money_precision_at_k(row['base_rec'], row['actual'], itemid_to_price, k=5), axis=1)
base=result_lvl_1['MP5'].mean()
base, als # Метрики money precision @ 5 до бизнесфильтра

(0.24102605504006505, 0.11678175055824593)

In [23]:
min([len(i) for  i in result_lvl_1['base_rec']]), min([len(i) for  i in result_lvl_1['als_rec']])
# Минимальное число item_id в рекомендациях

(500, 500)

In [24]:
# Фильтрация в соответствии с бизнес логикой

def postfilter(recommended_list,item_to_commodity, users_bought_list,prices, user_id, base_rec):
    print('user_id ', user_id)
    res = [] 
    com = []
    flag =0
    item = 0
    
    if user_id in top_more_4_for_user_dict.keys():
        top_more_4_for_user = top_more_4_for_user_dict[user_id]
    else: top_more_4_for_user = []
    
    if user_id in users_bought_list.keys():
        user_bought_list = users_bought_list[user_id]
    else: user_bought_list = []
        
    if user_id in users_purchases_frequency.keys():
        users_pu_fr = users_purchases_frequency[user_id][:300]
    else: users_pu_fr = []   
  
    #1
    for item in users_pu_fr[:55]+base_rec[:65]:
        if prices[item] > 7 : #and item in  users_pu_fr: # and item in recommended_list:#and item in user_bought_list: 
                print(' 1.1 ' , end='')
                res.append(item)
                com.append(item_to_commodity[item])
                if item in user_bought_list :
                    flag += 1 
                    print('i ' , end='')
                break
    if  len(res) == 0:
        for item in   recommended_list[:50]+base_rec[:50]+top_more_4_for_user[:50]:
            if prices[item] > 7:#users_pu_fr:#: # and item in recommended_list:#and item in user_bought_list:
                print(' 1.2 ', end='') 
                res.append(item)
                com.append(item_to_commodity[item])
                if item in user_bought_list :
                    flag += 1 
                    print('i ' , end='')
                break

    
    if   len(res) == 0:print('!!!!!!!!!!!!!!!!')
        
    # 2    
    for item in users_pu_fr[:75]+base_rec[:60]+top_frequency[:60]+recommended_list[:60]:
        if (item not in res) and (item_to_commodity[item] not in com) : #and item in  users_pu_fr: # and item in recommended_list:#and item in user_bought_list: 
                print(' 2.1 ' , end='')
                res.append(item)
                com.append(item_to_commodity[item])
                if item in user_bought_list :
                    flag += 1 
                    print('i ' , end='')
                break
    if  len(res) != 2:
        for item in  base_rec[:200]+recommended_list[:100]+top_more_4_for_user[:100] :
            if  (item not in res) and (item_to_commodity[item] not in com):#users_pu_fr:#top_frequency: # and item in recommended_list:#and item in user_bought_list:
                print(' 2.2 ', end='') 
                res.append(item)
                com.append(item_to_commodity[item])
                if item in user_bought_list :
                    flag += 1 
                    print('i ' , end='')
                break
 
    # 3    
    for item in  users_pu_fr[:80]+base_rec[:80]+top_frequency[:80]+recommended_list[:80]:
        if  (item not in res) and (item_to_commodity[item] not in com) : #and item in  users_pu_fr: # and item in recommended_list:#and item in user_bought_list: 
                print(' 3.1 ' , end='')
                res.append(item)
                com.append(item_to_commodity[item])
                if item in user_bought_list :
                    flag += 1 
                    print('i ' , end='')
                break
    if  len(res) != 3:
        for item in  top_frequency[:100]+recommended_list[:100]+base_rec[:100]+top_more_4_for_user[:100]:
            if (item not in res) and (item_to_commodity[item] not in com):#users_pu_fr:#top_frequency: # and item in recommended_list:#and item in user_bought_list:
                print(' 3.2 ', end='') 
                res.append(item)
                com.append(item_to_commodity[item])
                if item in user_bought_list :
                    flag += 1 
                    print('i ' , end='')
                break
                
    # 4    
    for item in users_pu_fr[:100]+top_frequency[:80]+recommended_list[:80]+base_rec[:80]:
        if prices[item] <1.35 and  (item not in res) and (item_to_commodity[item] not in com): # and item in recommended_list:#and item in user_bought_list: 
                if item in user_bought_list:
                    if flag<3:
                        flag +=1
                        print(' 4.1i ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break
                else:
                        print(' 4.1o ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break            
    if  len(res) != 4:
        for item in  top_frequency+recommended_list+base_rec :
            if prices[item] <1.2  and  (item not in res) and (item_to_commodity[item] not in com):
                if item in user_bought_list:
                    if flag<3:
                        flag +=1
                        print(' 4.2i ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break
                else:
                        print(' 4.2o ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break 
    # 5   
    for item in top_frequency[:200]+recommended_list[:100]+base_rec[:100]:
        if prices[item] <1.15 and  (item not in res) and (item_to_commodity[item] not in com): 
                if item in user_bought_list:
                    if flag<3:
                        flag +=1
                        print(' 5.1i ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break
                else:
                        print(' 5.1o ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break             
    if  len(res) != 5:
        for item in  top_frequency+base_rec+recommended_list :
            if prices[item] <1.2  and  (item not in res) and (item_to_commodity[item] not in com):
                if item in user_bought_list:
                    if flag<3:
                        flag +=1
                        print(' 5.2i ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break
                else:
                        print(' 5.2o ' , end='')
                        res.append(item)
                        com.append(item_to_commodity[item])
                        break 
    print()
    assert len(res) == 5, 'Количество рекомендаций != {}'.format(5)
    return res           

In [25]:
result_lvl_1['res'] = result_lvl_1.apply(lambda row:postfilter(row['als_rec'],item_to_commodity,
                                    users_bought_history,  itemid_to_price, row['user_id'],row['base_rec']), axis=1)

user_id  1
 1.2  2.1 i  3.1 i  4.1i  5.2o 
user_id  1
 1.2  2.1 i  3.1 i  4.1i  5.2o 
user_id  6
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  7
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  8
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  9
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  13
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  14
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  15
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  16
!!!!!!!!!!!!!!!!
 2.1 i  2.2 i  3.1 i  4.1o  5.1o 
user_id  17
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  18
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  19
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  20
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  22
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  23
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  24
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  25
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  26
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  27
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  28
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  29
 1.1 i  2.1 i  

user_id  279
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  280
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  282
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  283
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  285
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  286
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  287
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  288
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  289
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  290
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  291
 1.2  2.1 i  3.1 i  4.1i  5.2o 
user_id  292
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  293
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  294
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  297
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  300
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  301
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  302
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  303
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  304
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  305
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  306
 1.1 i

user_id  567
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  568
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  569
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  573
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  574
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  575
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  577
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  578
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  580
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  581
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  582
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  583
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  585
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  586
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  587
!!!!!!!!!!!!!!!!
 2.1 i  2.2 i  3.1 i  4.1o  5.1o 
user_id  588
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  590
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  591
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  592
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  593
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  594
 1.1 i  2.1 i  3.1 i  4.1o  5.

 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  911
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  913
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  914
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  915
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  916
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  919
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  920
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  922
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  923
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  924
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  925
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  926
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  927
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  928
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  929
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  932
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  933
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  934
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  935
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  936
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  937
 1.1 i  2.1 i  3.

user_id  1267
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1269
 1.2 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1270
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1271
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  1272
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1273
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1274
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1277
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1279
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1280
!!!!!!!!!!!!!!!!
 2.1 i  2.2  3.1 i  4.1o  5.1o 
user_id  1281
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  1282
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  1284
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1285
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1287
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1288
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1289
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1290
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1291
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1293
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1294
 1.1 i  2.1 i

 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1642
 1.2 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1643
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1644
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1647
 1.2  2.1 i  3.1 i  4.1o  5.1o 
user_id  1648
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1649
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1650
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1651
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1653
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1654
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1656
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1657
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1658
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1659
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1660
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1661
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1662
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  1663
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1664
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  1665
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  1

 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2023
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2024
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2025
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2026
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2027
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2028
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2030
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2031
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  2032
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2033
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2034
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2036
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2037
 1.2 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2038
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2039
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2040
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2041
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2043
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2044
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2047
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id 

 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2405
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  2406
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2407
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  2408
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2410
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2412
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2415
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2416
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2418
 1.2 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2419
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2420
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2421
 1.2 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2423
 1.2 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2424
 1.2 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  2425
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id  2426
 1.2 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2427
 1.2  2.1 i  3.1 i  4.1i  5.1o 
user_id  2430
 1.1 i  2.1 i  3.1 i  4.2o  5.2o 
user_id  2431
 1.1 i  2.1 i  3.1 i  4.1o  5.1o 
user_id  2432
 1.1 i  2.1 i  3.1 i  4.1o  5.2o 
user_id 

In [26]:
result_lvl_1['MP5'] = result_lvl_1.apply(lambda row: money_precision_at_k(row['res'], row['actual'], itemid_to_price, k=5), axis=1)
result_lvl_1['MP5'].mean()

0.190972082020397

In [27]:
result_lvl_1['_test_'] = result_lvl_1.apply(lambda row: res_list(row['res'], row['actual']), axis=1)


In [28]:
np.sum(result_lvl_1['_test_'])  # значения  = число угаданных item_id 

array([227, 853, 692, 151,  56])

In [29]:
#result_lvl_1['more_7'] = 0
result_lvl_1['more_7'] = result_lvl_1.apply(lambda row: price_more_7(row['res'],itemid_to_price), axis=1)
result_lvl_1['more_7'].sum()  # Сколько всего товаров >7

1891

In [30]:
# Посчитали количество новых товаров
result_lvl_1['too'] = result_lvl_1.apply(lambda row: more_2(row['res'],users_bought_history,row['user_id']), axis=1)

In [31]:
# Посчитали кол-во уникальных отделов в предсказании
result_lvl_1['dep'] = result_lvl_1.apply(lambda row: dep_5(row['res'],item_to_commodity), axis=1)

In [32]:
result_lvl_1

,user_id,actual,base_rec,als_rec,MP5,res,_test_,more_7,too,dep
0,1,"[883616, 917704, 931860, 961554, 995242, 10020...","[856942, 940947, 5577022, 934369, 9527290, 809...","[1047619, 896085, 1051211, 1075074, 948670, 86...",0.427916,"[948670, 856942, 940947, 1074612, 981760]","[0, 1, 1, 1, 0]",1,3,5
1,6,"[909479, 7431990, 6553035, 12263667, 13382461,...","[8203834, 13003092, 863447, 1098844, 9911484, ...","[878996, 1082185, 1024306, 930118, 871611, 857...",0.000000,"[12384953, 1082185, 863447, 833025, 1020682]","[0, 0, 0, 0, 0]",1,3,5
2,7,"[850841, 1085476, 12696020, 822346, 839747, 84...","[1122358, 828867, 1079023, 9338009, 6602729, 9...","[1127232, 839419, 1003188, 856335, 1127831, 92...",0.252004,"[974156, 1122358, 1082185, 981760, 840361]","[0, 1, 1, 0, 0]",1,3,5
3,8,"[884897, 893501, 1008074, 1051211, 6772795, 93...","[969932, 981660, 12302069, 972931, 1105433, 98...","[916122, 965766, 840361, 1068719, 1029743, 981...",0.088269,"[995660, 8181377, 969932, 1020682, 1134522]","[0, 1, 0, 0, 0]",1,3,5
4,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[862799, 1135468, 1056005, 1040346, 1039126, 1...","[923746, 1082185, 893018, 1029743, 1096036, 11...",0.107902,"[12171886, 1029743, 862799, 981760, 840361]","[0, 1, 0, 0, 0]",1,3,5
...,...,...,...,...,...,...,...,...,...,...
1894,2495,[1096603],"[867931, 906838, 1067419, 5568445, 839656, 981...","[839656, 916122, 876695, 917654, 867931, 11268...",0.000000,"[13007264, 867931, 1006184, 1082185, 840361]","[0, 0, 0, 0, 0]",1,3,5
1895,2496,[6534178],"[7441210, 995876, 820122, 1056509, 1000753, 12...","[844179, 1004906, 12810393, 916122, 1043751, 1...",0.000000,"[1000753, 916122, 5569230, 1082185, 833025]","[0, 0, 0, 0, 0]",1,3,5
1896,2497,"[820361, 884896, 896938, 938139, 965719, 99513...","[1135834, 1031864, 870515, 8090532, 849202, 94...","[5569230, 957951, 963971, 899624, 5569471, 844...",0.256074,"[849202, 1135834, 870515, 840361, 1044895]","[0, 1, 1, 0, 0]",1,3,5
1897,2498,"[32439, 73048, 386608, 404300, 15722019, 15722...","[1100379, 1076580, 1110031, 10456152, 5565356,...","[995242, 1053690, 1136257, 859075, 916122, 556...",0.000000,"[933354, 1070820, 961554, 981760, 886703]","[0, 0, 0, 0, 0]",1,3,5


In [33]:
result_lvl_1['more_7'].unique(), result_lvl_1['too'].unique(), result_lvl_1['dep'].unique()

(array([1, 0, 2], dtype=int64),
 array([3, 2, 1, 0], dtype=int64),
 array([5], dtype=int64))

Посчитаем на тесте

In [34]:
test = data_test.groupby('user_id')['item_id'].unique().reset_index()
test.columns=['user_id', 'actual']
test =  test.merge(result_lvl_1, on=['user_id'], how='left')
test.fillna(0, inplace=True)

In [35]:
test['MP5'] = test.apply(lambda row: money_precision_at_k(row['res'], row['actual_x'], itemid_to_price, k=5), axis=1)

In [36]:
test['MP5'].mean()

0.16868019857890218

In [ ]:
result = result_lvl_1['']

**Вывод. К сожалению перепробовав множество вариантов, вынужденно останавливаюсь на этом. Видимо что-то делаю не так.... Надеюсь разбор полетов поможет вникнуть в методологию решения задачи.**